# Building simulation for NET 01:
## Basic network, current situation
This notebook summarizes the simulation construction and running for net 01

In [1]:
#Make sure that the SUMO_HOME environment variable is correctly set after installation:
SUMO_HOME = r"C:\Program Files (x86)\Eclipse\Sumo"

In [6]:
# Defining network file
NET_FILE = ".\data/grid01-bcn_06.net.xml"

### Demand generation
Endogenous demand based on ACTIVITYGEN

In [2]:
## Config file for parsing demographic data:

ACTIVITYGEN_STAT_FILE = ".\data/activitygen-bcn_06.stat.xml"

## Basic trip output file (without routing) as a resukt of the demand generation based on ACTIVITYGEN.
TRIPS_FROM_ACTIVITYGEN_STAT = ".\data/activitygen-bcn_06.trips.rou.xml"

## Routes output files from ACTIVITYGEN trips
ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN = ".\data/activitygen-bcn_06.rou.xml"

In [3]:
# Calling ACTIVITYGEN tool in SUMO
# Random seed set for reproductibility purposes

!activitygen --net-file {NET_FILE} \
            --stat-file {ACTIVITYGEN_STAT_FILE} \
            --output-file {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --seed 25101987

first completed in getStreet() of City: Consolidation of data not needed in ActivityGen any more
--> work position: 
  |-> in city: 72055
  |-> out city: 79201
  |-> in+out city: 151256
--> population: 
  |-> city households: 21300
  |-> city people: 49065
    |-> city single: 0 / (in) couple: 41230
    |-> city adults: 41230
      |-> estimation: 43285
      |-> retired: 10322
    |-> city children: 7835
      |-> estimation: 7835
  |-> working people: 66987
    |-> working people in city: 29158
    |-> working people from outside: 37829
no problem during bus line trip generation...
after public transportation: 0
after household activities: 17757
no problem during in/out traffic generation...
after incoming/outgoing traffic: 93415
Before Random traffic generation (days are still entire):
- Total number of trips: 174546
- Total daily trips: 81131
- Total one-day trips: 12284
added uniform random trips: 9186
no problem during random traffic generation...
after random traffic: 102601
tot

In [ ]:
# Routing trips from ACTIVITYGEN
# CH routing algorithm is used

!duarouter --net-file {NET_FILE} \
            --route-files {TRIPS_FROM_ACTIVITYGEN_STAT} \
            --output-file {ROUTES_FROM_DUAROUTER_FROM_ACTIVITYGEN} \
            --routing-algorithm CH \
            --ignore-errors

Passing through traffic from random trips between fringe edges

In [ ]:
# creation of hourly distribution of trips based on ACTIVITYGEN output hourly histogram
dist = [209, 226, 263, 472, 1028, 2146, 3679, 5567, 6573, 6539, 5652, 4762, 4409, 4733, 5041, 5450, 6065, 6499, 6266, 5337, 3953, 2605, 1660, 1164]
total_random_trips = 280000
factor = total_random_trips/sum(dist)
random_dist = [int(element * factor) for element in dist]

In [ ]:
# Creating iteratively random trips accordingly to the hourly demand
## CREATE RANDOM FOLDER if it does not exist
import os
if not os.path.exists('data/random'):
    os.makedirs('data/random')

for i in range(len(random_dist)):
    begin = 3600*i
    end = 3600*(i+1)
    trips = random_dist[i]
    period = (end-begin)/trips
    trip_prefix = str(str(i) + "h")
    output_folder_1 = "data/random/random_h_"
    output_folder_2_trips = "-bcn_06.trips.rou.xml"
    output_folder_2_routes = "-bcn_06.rou.xml"

    BEGIN_ARG = str("-b " + str(begin))
    END_ARG = str("-e " + str(end))
    PERIOD_ARG = str("-p" + str(period))
    OUT_TRIPS_ARG = str("-o" + output_folder_1 + str(i) + output_folder_2_trips)
    OUT_ROUTES_ARG = str("-r" + output_folder_1 + str(i) + output_folder_2_routes)
    
    # bash SUMO command for RANDOM trips
    # Adjust the installation path for SUMO
    !python "$SUMO_HOME\tools\randomTrips.py" -n {NET_FILE} \
    --prefix={trip_prefix} \
    --fringe-factor 100 \
    {BEGIN_ARG} \
    {END_ARG} \
    {PERIOD_ARG} \
    --binomial 10 \
    "{OUT_TRIPS_ARG}" \
    --edge-permission=passenger

In [ ]:
# Merging all random routing files i|n a single file
import os
list_trips_files = []
for file in os.listdir(r"data/random/"):
    if file.endswith(".trips.rou.xml"):
        list_trips_files.append(str("data/random/" + file))
list_trips_files_str = ' '.join([str(str(elem) + ',') for elem in list_trips_files])
list_trips_files_str = list_trips_files_str.replace(" ","")

SINGLE_RANDOM_ROUTES_FILE = "data/random/random_ALL.rou.xml"

!duarouter --net-file {NET_FILE} \
--route-files {list_trips_files_str[0:-1]} \
--output-file {SINGLE_RANDOM_ROUTES_FILE} \
--routing-algorithm CH \
--ignore-errors

### Microscopic Simulation
#### Setting configuration files
A XML configuration file (**data/compar_002.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file.xml**) for results colletion.

In [8]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder
import os, time
if not os.path.exists('data/results'):
    os.makedirs('data/results')

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output data/results/overall_stats.xml'.format(scale)
    SUMMARY_ARG = '--summary data/results/summary.xml'.format(scale)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_002.sumo.cfg \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1 \
    -W
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

STARTING sumo simulation with scale 2.00
Loading configuration ... done.
Step #0.00 (1ms ~= 1000.00*RT, ~0.00UPS, vehicles TOT 0 ACT 0 BUF 0)                      
Step #100.00 (1ms ~= 1000.00*RT, ~32000.00UPS, vehicles TOT 32 ACT 32 BUF 2)              
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 59 ACT 53 BUF 1)                               
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 98 ACT 79 BUF 0)                               
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 128 ACT 82 BUF 0)                              
Step #500.00 (1ms ~= 1000.00*RT, ~92000.00UPS, vehicles TOT 165 ACT 92 BUF 1)             
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 196 ACT 85 BUF 0)                              
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 217 ACT 72 BUF 3)                              
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 258 ACT 89 BUF 0)                              
Step #900.00 (2ms ~= 500.00*RT, ~49500.00UPS, vehicles TOT 296 ACT 99 BUF 0)              
Step #1000.00 (0m

Step #17700.00 (6ms ~= 166.67*RT, ~111333.33UPS, vehicles TOT 14699 ACT 668 BUF 3)        
Step #17800.00 (4ms ~= 250.00*RT, ~158750.00UPS, vehicles TOT 14920 ACT 635 BUF 2)        
Step #17900.00 (5ms ~= 200.00*RT, ~147400.00UPS, vehicles TOT 15190 ACT 737 BUF 16)       
Step #18000.00 (7ms ~= 142.86*RT, ~109285.71UPS, vehicles TOT 15456 ACT 765 BUF 8)        
Step #18100.00 (9ms ~= 111.11*RT, ~104333.33UPS, vehicles TOT 15861 ACT 939 BUF 23)       
Step #18200.00 (8ms ~= 125.00*RT, ~133500.00UPS, vehicles TOT 16287 ACT 1068 BUF 35)      
Step #18300.00 (12ms ~= 83.33*RT, ~98833.33UPS, vehicles TOT 16737 ACT 1186 BUF 45)       
Step #18400.00 (10ms ~= 100.00*RT, ~132600.00UPS, vehicles TOT 17171 ACT 1326 BUF 45)     
Step #18500.00 (18ms ~= 55.56*RT, ~78277.78UPS, vehicles TOT 17624 ACT 1409 BUF 64)       
Step #18600.00 (16ms ~= 62.50*RT, ~88375.00UPS, vehicles TOT 18042 ACT 1414 BUF 46)       
Step #18700.00 (12ms ~= 83.33*RT, ~116000.00UPS, vehicles TOT 18515 ACT 1392 BUF 57)      

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (1ms ~= 1000.00*RT, ~31000.00UPS, vehicles TOT 31 ACT 31 BUF 2)              
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 57 ACT 51 BUF 1)                               
Step #300.00 (1ms ~= 1000.00*RT, ~76000.00UPS, vehicles TOT 95 ACT 76 BUF 0)              
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 124 ACT 80 BUF 0)                              
Step #500.00 (1ms ~= 1000.00*RT, ~89000.00UPS, vehicles TOT 160 ACT 89 BUF 1)             
Step #600.00 (1ms ~= 1000.00*RT, ~82000.00UPS, vehicles TOT 191 ACT 82 BUF 0)             
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 211 ACT 69 BUF 3)                              
Step #800.00 (1ms ~= 1000.00*RT, ~85000.00UPS, vehicles TOT 251 ACT 85 BUF 0)             
Step #900.00 (1ms ~= 1000.00*RT, ~92000.00UPS, vehicles TOT 288 ACT 92 BUF 0)             
Step #1000.00 (1ms ~= 1000.00*RT, ~92000.00UPS, vehicles T

Step #14100.00 (2ms ~= 500.00*RT, ~125000.00UPS, vehicles TOT 7290 ACT 250 BUF 0)         
Step #14200.00 (2ms ~= 500.00*RT, ~129000.00UPS, vehicles TOT 7386 ACT 258 BUF 2)         
Step #14300.00 (3ms ~= 333.33*RT, ~98666.67UPS, vehicles TOT 7498 ACT 296 BUF 3)          
Step #14400.00 (4ms ~= 250.00*RT, ~76750.00UPS, vehicles TOT 7582 ACT 307 BUF 3)          
Step #14500.00 (3ms ~= 333.33*RT, ~123333.33UPS, vehicles TOT 7749 ACT 370 BUF 6)         
Step #14600.00 (3ms ~= 333.33*RT, ~146000.00UPS, vehicles TOT 7941 ACT 438 BUF 5)         
Step #14700.00 (5ms ~= 200.00*RT, ~106000.00UPS, vehicles TOT 8160 ACT 530 BUF 6)         
Step #14800.00 (4ms ~= 250.00*RT, ~137000.00UPS, vehicles TOT 8336 ACT 548 BUF 8)         
Step #14900.00 (8ms ~= 125.00*RT, ~75875.00UPS, vehicles TOT 8535 ACT 607 BUF 15)         
Step #15000.00 (6ms ~= 166.67*RT, ~97333.33UPS, vehicles TOT 8726 ACT 584 BUF 4)          
Step #15100.00 (5ms ~= 200.00*RT, ~113400.00UPS, vehicles TOT 8912 ACT 567 BUF 3)         

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 30 ACT 30 BUF 2)                               
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 56 ACT 49 BUF 1)                               
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 93 ACT 75 BUF 0)                               
Step #400.00 (1ms ~= 1000.00*RT, ~79000.00UPS, vehicles TOT 121 ACT 79 BUF 0)             
Step #500.00 (1ms ~= 1000.00*RT, ~89000.00UPS, vehicles TOT 156 ACT 89 BUF 1)             
Step #600.00 (1ms ~= 1000.00*RT, ~79000.00UPS, vehicles TOT 186 ACT 79 BUF 0)             
Step #700.00 (1ms ~= 1000.00*RT, ~67000.00UPS, vehicles TOT 206 ACT 67 BUF 3)             
Step #800.00 (1ms ~= 1000.00*RT, ~83000.00UPS, vehicles TOT 245 ACT 83 BUF 0)             
Step #900.00 (1ms ~= 1000.00*RT, ~92000.00UPS, vehicles TOT 281 ACT 92 BUF 0)             
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 308 ACT 91 BUF

...

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 1 ACT 1 BUF 0)                                 
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 3 ACT 3 BUF 0)                                 
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 4 BUF 0)                                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 3 BUF 0)                                 
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 9 ACT 5 BUF 0)                                 
Step #600.00 (0ms ?*RT. ?UPS, vehicles TOT 10 ACT 4 BUF 0)                                
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 11 ACT 4 BUF 0)                                
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 13 ACT 5 BUF 0)                                
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 15 ACT 5 BUF 0)                                
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 16 ACT 3 BUF 0

Step #26700.00 (2ms ~= 500.00*RT, ~86500.00UPS, vehicles TOT 3908 ACT 173 BUF 0)          
Step #26800.00 (1ms ~= 1000.00*RT, ~178000.00UPS, vehicles TOT 3967 ACT 178 BUF 0)        
Step #26900.00 (1ms ~= 1000.00*RT, ~182000.00UPS, vehicles TOT 4026 ACT 182 BUF 0)        
Step #27000.00 (3ms ~= 333.33*RT, ~63333.33UPS, vehicles TOT 4089 ACT 190 BUF 0)          
Step #27100.00 (1ms ~= 1000.00*RT, ~186000.00UPS, vehicles TOT 4148 ACT 186 BUF 0)        
Step #27200.00 (1ms ~= 1000.00*RT, ~175000.00UPS, vehicles TOT 4208 ACT 175 BUF 0)        
Step #27300.00 (4ms ~= 250.00*RT, ~41750.00UPS, vehicles TOT 4269 ACT 167 BUF 0)          
Step #27400.00 (1ms ~= 1000.00*RT, ~176000.00UPS, vehicles TOT 4330 ACT 176 BUF 0)        
Step #27500.00 (2ms ~= 500.00*RT, ~95000.00UPS, vehicles TOT 4390 ACT 190 BUF 0)          
Step #27600.00 (2ms ~= 500.00*RT, ~92000.00UPS, vehicles TOT 4450 ACT 184 BUF 0)          
Step #27700.00 (1ms ~= 1000.00*RT, ~182000.00UPS, vehicles TOT 4511 ACT 182 BUF 0)        

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                   
Step #100.00 (0ms ?*RT. ?UPS, vehicles TOT 0 ACT 0 BUF 0)                                 
Step #200.00 (0ms ?*RT. ?UPS, vehicles TOT 2 ACT 2 BUF 0)                                 
Step #300.00 (0ms ?*RT. ?UPS, vehicles TOT 3 ACT 3 BUF 0)                                 
Step #400.00 (0ms ?*RT. ?UPS, vehicles TOT 4 ACT 3 BUF 0)                                 
Step #500.00 (0ms ?*RT. ?UPS, vehicles TOT 5 ACT 3 BUF 0)                                 
Step #600.00 (1ms ~= 1000.00*RT, ~1000.00UPS, vehicles TOT 5 ACT 1 BUF 0)                 
Step #700.00 (0ms ?*RT. ?UPS, vehicles TOT 6 ACT 2 BUF 0)                                 
Step #800.00 (0ms ?*RT. ?UPS, vehicles TOT 7 ACT 2 BUF 0)                                 
Step #900.00 (0ms ?*RT. ?UPS, vehicles TOT 8 ACT 2 BUF 0)                                 
Step #1000.00 (0ms ?*RT. ?UPS, vehicles TOT 8 ACT 1 BUF 0)


Step #13200.00 (0ms ?*RT. ?UPS, vehicles TOT 166 ACT 5 BUF 0)                             
Step #13300.00 (0ms ?*RT. ?UPS, vehicles TOT 168 ACT 6 BUF 0)                             
Step #13400.00 (0ms ?*RT. ?UPS, vehicles TOT 171 ACT 7 BUF 0)                             
Step #13500.00 (0ms ?*RT. ?UPS, vehicles TOT 174 ACT 8 BUF 0)                             
Step #13600.00 (0ms ?*RT. ?UPS, vehicles TOT 175 ACT 9 BUF 0)                             
Step #13700.00 (0ms ?*RT. ?UPS, vehicles TOT 178 ACT 10 BUF 0)                            
Step #13800.00 (0ms ?*RT. ?UPS, vehicles TOT 180 ACT 7 BUF 0)                             
Step #13900.00 (0ms ?*RT. ?UPS, vehicles TOT 183 ACT 9 BUF 0)                             
Step #14000.00 (1ms ~= 1000.00*RT, ~7000.00UPS, vehicles TOT 185 ACT 7 BUF 0)             
Step #14100.00 (1ms ~= 1000.00*RT, ~7000.00UPS, vehicles TOT 187 ACT 7 BUF 0)             
Step #14200.00 (0ms ?*RT. ?UPS, vehicles TOT 190 ACT 7 BUF 0)                            

Step #69800.00 (1ms ~= 1000.00*RT, ~71000.00UPS, vehicles TOT 13750 ACT 71 BUF 0)         
Step #69900.00 (1ms ~= 1000.00*RT, ~70000.00UPS, vehicles TOT 13774 ACT 70 BUF 0)         
Step #70000.00 (1ms ~= 1000.00*RT, ~68000.00UPS, vehicles TOT 13798 ACT 68 BUF 0)         
Step #70100.00 (1ms ~= 1000.00*RT, ~71000.00UPS, vehicles TOT 13819 ACT 71 BUF 0)         
Step #70200.00 (2ms ~= 500.00*RT, ~36000.00UPS, vehicles TOT 13844 ACT 72 BUF 0)          
Step #70300.00 (0ms ?*RT. ?UPS, vehicles TOT 13866 ACT 69 BUF 0)                          
Step #70400.00 (0ms ?*RT. ?UPS, vehicles TOT 13889 ACT 61 BUF 0)                          
Step #70500.00 (1ms ~= 1000.00*RT, ~69000.00UPS, vehicles TOT 13913 ACT 69 BUF 0)         
Step #70600.00 (1ms ~= 1000.00*RT, ~73000.00UPS, vehicles TOT 13937 ACT 73 BUF 0)         
Step #70700.00 (0ms ?*RT. ?UPS, vehicles TOT 13959 ACT 69 BUF 0)                          
Step #70800.00 (0ms ?*RT. ?UPS, vehicles TOT 13984 ACT 60 BUF 0)                          

### Mesoscopic Simulations
#### Setting configuration files
A XML configuration file (**data/compar_002_meso.sumo.cfg**) is set for taking all the demand inputs, the net, and specifying the output files (in configuration file **data/init_edgedata_additional_file_meso.xml**) for results collection.

Three different calibrations are finally run for the mesoscopic case, by changing the headway times ($\tau values$) and other mesoscopic parameters as follows

#### Mesoscopic simulation trial01

Param:

$\tau_{ff} = 1$

$\tau_{fj} = 1.2$

$\tau_{jf} = 1.8$

$\tau_{jj} = 1.4$

--meso-tls-penalty = 2

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_01 = r'data/results_meso_trial01'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_01):
    os.makedirs(RESULTS_FOLDER_MESO_01)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_01)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_01)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_002_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 1 \
    --meso-taufj 1.2 \
    --meso-taujf 1.8 \
    --meso-taujj 1.4 \
    --meso-lane-queue \
    --meso-tls-penalty 2 \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial02

Param:

$\tau_{ff} = 2.1$

$\tau_{fj} = 0.75$

$\tau_{jf} = 0.75$

$\tau_{jj} = 0.5$

--meso-jam-threshold = -0.5

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_02 = r'data/results_meso_trial02'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_02):
    os.makedirs(RESULTS_FOLDER_MESO_02)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_02)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_02)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_002_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-tauff 2.1 \
    --meso-taufj 0.75 \
    --meso-taujf 0.75 \
    --meso-taujj 0.5 \
    --meso-lane-queue \
    --meso-jam-threshold -0.5 \
    --meso-junction-control \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value

#### Mesoscopic simulation trial03

Params:

$\tau_{ff} = 1.13$

$\tau_{fj} = 1.13$

$\tau_{jf} = 1.73$

$\tau_{jj} = 1.4$

--meso-jam-threshold = -1

--meso-junction-control True

In [ ]:
# Run simulation
## A scaling factor is introduced to tests 40 different scenarios with increasing traffic demand.

#create results folder

RESULTS_FOLDER_MESO_03 = r'data/results_meso_trial03'

import os, time
if not os.path.exists(RESULTS_FOLDER_MESO_03):
    os.makedirs(RESULTS_FOLDER_MESO_03)

total_veh_ref = 315000 # number of vehicles for scale factor 1.0
scale = 2

# running a loop from scale factor 2.0 to scale factor 0.05 and writing the results files.
while scale > 0.0:
    start = time.time()
    print('STARTING sumo simulation with scale {:04.2f}'.format(scale))
    scaled_veh_ref = total_veh_ref * scale
    max_num_teleports = int(scaled_veh_ref * 0.005) #value for stopping the simulation at that iteration
    OUTPUT_PREFIX_ARG = '--output-prefix scale_{:04.2f}_'.format(scale)
    STAT_OUT_ARG = '--statistic-output {}/overall_stats.xml'.format(RESULTS_FOLDER_MESO_03)
    SUMMARY_ARG = '--summary {}/summary.xml'.format(RESULTS_FOLDER_MESO_03)
    MAX_TELEP_ARG = '--max-num-teleports {:d}'.format(max_num_teleports)
    SCALE_ARG = '--scale {:f}'.format(scale)
    
    # running the simulation
    # ref https://sumo.dlr.de/docs/sumo.html
    !sumo -c .\data/compar_002_meso.sumo.cfg \
    --mesosim \
    --meso-edgelength 135 \
    --meso-junction-control \
    --meso-lane-queue \
    --meso-overtaking \
    {OUTPUT_PREFIX_ARG} \
    {STAT_OUT_ARG} \
    {SUMMARY_ARG} \
    {MAX_TELEP_ARG} \
    {SCALE_ARG} \
    --threads 1
    
    # measuring time for performance
    end = time.time()
    elapsed_time = (end-start)/60
    
    print('\nDONE sumo simulation with scale {:04.2f} in {:04.1f} minutes \n'.format(scale, elapsed_time))
    
    scale -= 0.05 #updating scale value